# `rules` are magic functions and regular expressions that express unique displays from strings.

In [3]:
import abc, fnmatch, IPython, functools

In [4]:
import vdom; from vdom import div, img; from vdom.svg import iframe
from mimetypes import guess_type; guess = lambda x: guess_type(x)[0]

In [5]:
     import IPython

In [6]:
    ip = get_ipython()

In [7]:
    from dataclasses import dataclass

In [8]:
    import inspect

In [9]:
    import operator

In [10]:
    @dataclass
    class NoArgsCellMagic:
        callable: callable = None
        __name__ = property(operator.attrgetter('callable.__name__'))
        __doc__ = property(operator.attrgetter('callable.__doc__'))
        def __call__(self, line, cell): return self.callable(cell)

In [11]:
    import mistune

In [16]:
    class MagicStringsDispatch(abc.ABCMeta):
        
        def append(self, key):
            ip = IPython.get_ipython()
            def attach(callable): 
                ip.register_magic_function(NoArgsCellMagic(callable), 'cell')
                setattr(self, key, callable)
                return callable
            return attach
        

        def __call__(self, str, result=None):
            for attr in (x for x in dir(self) if '*' in x):
                if fnmatch.fnmatch(str, attr):
                    result = getattr(self, attr)(str)
                    
                    if isinstance(result, dict) and str in result:
                        result = result[str]
                        if result['status'] == 'ok':
                            return result['data'], result['metadata']
                        else: ...

                    elif result is not None:
                        data, metadata = IPython.get_ipython().display_formatter.format(result)
                        return data, metadata
                    
                
            if str and str.splitlines()[0].strip():
                return {
                    'text/markdown': str
                }, {}

In [13]:
    class MagicStrings(metaclass=MagicStringsDispatch): ...

In [14]:
def load_ipython_extension(ip):
    ip.display_formatter.mimebundle_formatter.for_type(str, MagicStrings)
    
def unload_ipython_extension(ip):
    ip.display_formatter.mimebundle_formatter.for_type(str)
    
__name__ == '__main__' and load_ipython_extension(get_ipython())